In [12]:
from time import time
import polars as pl
import pandas as pd
import numpy as np
import pyarrow

import matplotlib.pyplot as plt
import seaborn as sns

df = pl.read_csv(r"./datasets/2019-Nov-1M.csv")

In [13]:
%%time

print(f"Dataset Shape: {df.shape}")
print(f"Dataset rows: {df.height}")
print(f"Dataset columns: {df.width}")

print(f"\nColumn Types\n-----")

for idx, col in enumerate(df.columns):
   print(f"{col}\tType: {df.dtypes[idx]}")

Dataset Shape: (1000000, 9)
Dataset rows: 1000000
Dataset columns: 9

Column Types
-----
event_time	Type: Utf8
event_type	Type: Utf8
product_id	Type: Int64
category_id	Type: Int64
category_code	Type: Utf8
brand	Type: Utf8
price	Type: Float64
user_id	Type: Int64
user_session	Type: Utf8
CPU times: total: 0 ns
Wall time: 0 ns


In [14]:
%%time

# Data Inspection

df.head(5)

CPU times: total: 0 ns
Wall time: 0 ns


event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
str,str,i64,i64,str,str,f64,i64,str
"""2019-11-01 00:…","""view""",1003461,2053013555631882655,"""electronics.sm…","""xiaomi""",489.07,520088904,"""4d3b30da-a5e4-…"
"""2019-11-01 00:…","""view""",5000088,2053013566100866035,"""appliances.sew…","""janome""",293.65,530496790,"""8e5f4f83-366c-…"
"""2019-11-01 00:…","""view""",17302664,2053013553853497655,null,"""creed""",28.31,561587266,"""755422e7-9040-…"
"""2019-11-01 00:…","""view""",3601530,2053013563810775923,"""appliances.kit…","""lg""",712.87,518085591,"""3bfb58cd-7892-…"
"""2019-11-01 00:…","""view""",1004775,2053013555631882655,"""electronics.sm…","""xiaomi""",183.27,558856683,"""313628f1-68b8-…"


In [16]:
df = df.with_columns(
    (pl.col("event_time").str.strptime(pl.Datetime, format="%Y-%m-%d %H:%M:%S %Z"))
)

df.head(15).select("event_time")

SchemaError: invalid series dtype: expected `Utf8`, got `datetime[μs]`

event_time
datetime[μs]
2019-11-01 00:00:00
2019-11-01 00:00:00
2019-11-01 00:00:01
2019-11-01 00:00:01
2019-11-01 00:00:01
2019-11-01 00:00:01
2019-11-01 00:00:01
2019-11-01 00:00:02
2019-11-01 00:00:02
